In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error
                             ,mean_absolute_error)
from sklearn import metrics

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.


In [2]:
traindata = pd.read_csv('/Users/yusufsatilmis/Desktop/Bilgisayar_Aglari/Training.csv', header=None)
testdata = pd.read_csv('/Users/yusufsatilmis/Desktop/Bilgisayar_Aglari/Testing.csv', header=None)

In [3]:
X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,0]
C = testdata.iloc[:,0]
T = testdata.iloc[:,1:42]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)

In [4]:
X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))

lstm_output_size = 70

cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(41, 1)))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(LSTM(lstm_output_size))
cnn.add(Dropout(0.1))
cnn.add(Dense(1, activation="sigmoid"))

/anaconda2/envs/deeplearning2/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same", activation="relu", input_shape=(41, 1))`
  import sys
/anaconda2/envs/deeplearning2/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  


In [5]:
cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

# train
checkpointer = callbacks.ModelCheckpoint(filepath="results/cnn1results/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('results/cnn1results/cnntrainanalysis1.csv',separator=',', append=False)
cnn.fit(X_train, y_train, nb_epoch=10,validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("results/cnn1results/cnn_model.hdf5")

/anaconda2/envs/deeplearning2/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 493715 samples, validate on 306 samples
Epoch 1/10
493715/493715 [==============================] - 717s 1ms/step - loss: 0.0297 - acc: 0.9900 - val_loss: 0.0076 - val_acc: 0.9967

Epoch 00001: val_acc improved from -inf to 0.99673, saving model to results/cnn1results/checkpoint-01.hdf5
Epoch 2/10
493715/493715 [==============================] - 684s 1ms/step - loss: 0.0128 - acc: 0.9959 - val_loss: 0.0180 - val_acc: 0.9935

Epoch 00002: val_acc did not improve from 0.99673
Epoch 3/10
493715/493715 [==============================] - 684s 1ms/step - loss: 0.0098 - acc: 0.9969 - val_loss: 0.0201 - val_acc: 0.9935

Epoch 00003: val_acc did not improve from 0.99673
Epoch 4/10
493715/493715 [==============================] - 686s 1ms/step - loss: 0.0073 - acc: 0.9976 - val_loss: 0.0199 - val_acc: 0.9935

Epoch 00004: val_acc did not improve from 0.99673
Epoch 5/10
493715/493715 [==============================] - 696s 1ms/step - loss: 0.0061 - acc: 0.9980 - val_loss: 0.0209 - val_ac

In [ ]:
cnn.load_weights("results/cnn1results/checkpoint-947.hdf5")


cnn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [6]:
loss, accuracy = cnn.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))

306/306 [==============================] - 0s 513us/step

Loss: 0.00, Accuracy: 99.67%


In [10]:
y_pred = cnn.predict_classes(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average="binary")
precision = precision_score(y_test, y_pred , average="binary")
f1 = f1_score(y_test, y_pred, average="binary")
np.savetxt('results/expected1.txt', y_test, fmt='%01d')
np.savetxt('results/predicted1.txt', y_pred, fmt='%01d')

print("confusion matrix")
print("----------------------------------------------")
print("accuracy")
print("%.6f" %accuracy)
print("racall")
print("%.6f" %recall)
print("precision")
print("%.6f" %precision)
print("f1score")
print("%.6f" %f1)
cm = metrics.confusion_matrix(y_test, y_pred)
print("==============================================")
print (cm)

confusion matrix
----------------------------------------------
accuracy
0.996732
racall
0.000000
precision
0.000000
f1score
0.000000
[[305   1]
 [  0   0]]
